In [154]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from sklearn import preprocessing
from functools import reduce
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

In [155]:
# Data cleaning function

def Country_Swap(target, acquirer):
    """Makes initial replacements of certain terms for eventual fuzzy matching and then fuzzy matches country names to improve dataframe merge results."""
    Swap_Out = ['Korea (Rep. of)', 'Democratic', 'Republic', 'Czechia', 'Slovakia', 'West Bank and Gaza', 'Kyrgyzstan', 'T.F.Y.R. Macedonia', 'Brunei Darussalam', 'Syrian Arab Rep.']
    Swap_In = ['Korea', 'Dem.', 'Rep.', 'Czech Rep.', 'Slovak Rep.', 'Palestine', 'Kyrgyz Rep.', 'North Macedonia', 'Brunei', 'Syria']
    for i in Swap_Out:
        acquirer['Country'] = acquirer['Country'].str.replace(i, Swap_In[Swap_Out.index(i)])
        target['Country'] = target['Country'].str.replace(i, Swap_In[Swap_Out.index(i)])
        
    banned_list = ['Australia', 'Austria', 'Iceland', 'Ireland', 'Congo', 'China']
    target_list = list(target['Country'].unique())
    acquirer_list = list(acquirer['Country'].unique())
    
    for i in target_list:
        for j in acquirer_list:
            if i not in acquirer_list:
                if (fuzz.ratio(i,j) >= 80) and (i not in banned_list) and (j not in banned_list):
                    target['Country'] = target['Country'].replace(to_replace = i, value = j)
                elif (fuzz.partial_ratio(i,j) >= 90) and (i not in banned_list) and (j not in banned_list):
                    target['Country'] = target['Country'].replace(to_replace = i, value = j)
                elif (fuzz.token_set_ratio(i,j) >= 90) and (i not in banned_list) and (j not in banned_list):
                    target['Country'] = target['Country'].replace(to_replace = i, value = j)
    return target, acquirer

In [156]:
# Creating dataframe for comparison of gdp and internet use and CO2 emissions.

gdp_and_internet_use = pd.read_csv('../data/gdp_and_internet_use_to_be_imported_and_merged.csv')
gdp_and_internet_use.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3873 entries, 0 to 3872
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country           3873 non-null   object 
 1   Year              3873 non-null   int64  
 2   GDP_Per_Capita    3873 non-null   float64
 3   Continent         3873 non-null   object 
 4   Internet_Use_Pct  3873 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 151.4+ KB


In [157]:
emissions = pd.read_csv("../data/CO2 Emissions.csv", names = ['Country', 'Year', 'CO2 (kts) Emitted'], header = 0)
emissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247 entries, 0 to 1246
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Country            1247 non-null   object 
 1   Year               1247 non-null   int64  
 2   CO2 (kts) Emitted  1247 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 29.4+ KB


In [158]:
outer_test_merge = pd.merge(gdp_and_internet_use, emissions, how = 'outer')
outer_list = list(outer_test_merge['Country'].unique())
inner_test_merge = pd.merge(gdp_and_internet_use, emissions, how = 'inner')
inner_list = list(inner_test_merge['Country'].unique())
for i in outer_list:
    if i not in inner_list:
        flag = True
        message = "These do not match. Checking for missing values."
if flag == True:
    print(message)

These do not match. Checking for missing values.


In [159]:
emissions, gdp_and_internet_use = Country_Swap(emissions, gdp_and_internet_use)
gaiu_emissions = pd.merge(gdp_and_internet_use, emissions, how = 'inner')
gaiu_emissions

,Country,Year,GDP_Per_Capita,Continent,Internet_Use_Pct,CO2 (kts) Emitted
0,Australia,2014,47436.748390,Oceania,84.560000,3.941169e+05
1,Australia,2013,46960.050240,Oceania,83.000000,3.979432e+05
2,Australia,2012,46571.474558,Oceania,79.000000,4.065062e+05
3,Australia,2011,45604.797159,Oceania,79.487698,4.041728e+05
4,Australia,2010,45131.434148,Oceania,76.000000,4.055028e+05
...,...,...,...,...,...,...
945,United States,1994,42410.326048,North America,4.862781,5.377033e+06
946,United States,1993,41270.867304,North America,2.271673,5.283438e+06
947,United States,1992,40698.340183,North America,1.724203,5.182707e+06
948,United States,1991,39862.575565,North America,1.163194,5.078890e+06


In [160]:
gaiu_emissions = gaiu_emissions.rename(columns = {'GDP_Per_Capita':'GDP Per Capita', 'Internet_Use_Pct':'% Internet Use'})
gaiu_emissions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 950 entries, 0 to 949
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Country            950 non-null    object 
 1   Year               950 non-null    int64  
 2   GDP Per Capita     950 non-null    float64
 3   Continent          950 non-null    object 
 4   % Internet Use     950 non-null    float64
 5   CO2 (kts) Emitted  950 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 52.0+ KB


In [161]:
gaiu_emissions_2014 = gaiu_emissions[gaiu_emissions['Year'] == 2014]
gaiu_emissions_1990 = gaiu_emissions[gaiu_emissions['Year'] == 1990]

In [162]:
relevant_variables = ['GDP Per Capita', '% Internet Use', 'CO2 (kts) Emitted']

min_max_scaler = preprocessing.MinMaxScaler()
for i in relevant_variables:
    gaiu_emissions_2014[i] = min_max_scaler.fit_transform(gaiu_emissions_2014[[i]])
    
min_max_scaler = preprocessing.MinMaxScaler()
for i in relevant_variables:
    gaiu_emissions_1990[i] = min_max_scaler.fit_transform(gaiu_emissions_1990[[i]])
    
gaiu_emissions_1990_2014 = pd.concat([gaiu_emissions_1990, gaiu_emissions_2014])

In [163]:
gaiu_emissions.to_csv('../data/gaiu_emissions_to_be_imported.csv', index = False)
gaiu_emissions_1990_2014.to_csv('../data/gaiu_emissions_1990_2014_to_be_imported.csv', index = False)

In [164]:
# Adding income inequality measures to GDP per capita dataframe

gdp_df = pd.read_csv('../data/gdp_df_to_be_imported_and_merged.csv')
gdp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5443 entries, 0 to 5442
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         5443 non-null   object 
 1   Year            5443 non-null   int64  
 2   GDP_Per_Capita  5443 non-null   float64
 3   Continent       5443 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 170.2+ KB


In [165]:
def Take_Out_the_NAs_On_Read(values):
    """Drops NA values on csv read."""
    if values == np.nan:
        return 0
    return values

merged_df_1 = pd.read_csv("../data/Top Ten Income Share.csv", skipfooter = 253, usecols = [i for i in range(3)])
merged_df_2 = pd.read_csv("../data/Poverty Headcount $5.50.csv", skipfooter = 260, usecols = [i for i in range(3)], converters={"Value": Take_Out_the_NAs_On_Read})
merged_df_3 = pd.read_csv("../data/Poverty Headcount $1.90.csv", skipfooter = 261, usecols = [i for i in range(3)], converters={"Value": Take_Out_the_NAs_On_Read})

In [166]:
to_be_merged_dataframes = [gdp_df, merged_df_1, merged_df_2, merged_df_3]

for i in to_be_merged_dataframes:
    if not i.equals(gdp_df):
        i.rename(columns = {"Country or Area":"Country"}, inplace = True)
        i['Year'] = i['Year'].astype(int)
        i['Value'] = i['Value'].astype(np.float)
        Country_Swap(i, gdp_df)

In [167]:
merged_df_1.rename(columns = {"Value":"% Income Held by Top 10%"}, inplace = True)
merged_df_2.rename(columns = {"Value":"% Population Living on <$5.50/Day"}, inplace = True)
merged_df_3.rename(columns = {"Value":"% Population Living on <$1.90/Day"}, inplace = True)

In [168]:
fully_merged = reduce(lambda left, right: 
                      pd.merge(left, right, on = ['Country', 'Year'], how = 'inner'),
                      to_be_merged_dataframes)

In [169]:
fully_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1638 entries, 0 to 1637
Data columns (total 7 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Country                            1638 non-null   object 
 1   Year                               1638 non-null   int64  
 2   GDP_Per_Capita                     1638 non-null   float64
 3   Continent                          1638 non-null   object 
 4   % Income Held by Top 10%           1638 non-null   float64
 5   % Population Living on <$5.50/Day  1638 non-null   float64
 6   % Population Living on <$1.90/Day  1638 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 102.4+ KB


In [170]:
fully_merged.head()

,Country,Year,GDP_Per_Capita,Continent,% Income Held by Top 10%,% Population Living on <$5.50/Day,% Population Living on <$1.90/Day
0,Albania,2017,12811.759436,Europe,24.8,33.8,1.3
1,Albania,2016,12331.127754,Europe,25.0,34.2,0.9
2,Albania,2015,11916.422315,Europe,24.9,34.3,1.1
3,Albania,2014,11623.866679,Europe,25.5,37.0,1.6
4,Albania,2012,11263.851342,Europe,22.9,34.5,0.8


In [171]:
fully_merged.tail()

,Country,Year,GDP_Per_Capita,Continent,% Income Held by Top 10%,% Population Living on <$5.50/Day,% Population Living on <$1.90/Day
1633,Zambia,1996,1974.616550,Africa,37.3,84.8,43.6
1634,Zambia,1993,2133.263108,Africa,39.3,89.0,56.2
1635,Zambia,1991,2133.689042,Africa,42.9,85.1,55.2
1636,Zimbabwe,2017,3028.245976,Africa,34.8,81.3,33.9
1637,Zimbabwe,2011,2556.278431,Africa,33.8,74.0,21.4


In [172]:
fully_merged.to_csv('../data/fully_merged_to_be_imported.csv', index = False)